In [94]:
import pandas as pd
import torch
import random
import numpy as np
from torch.autograd import Variable

In [95]:
def get_data(fname):
    df = pd.read_csv(fname)
    df = df.drop(columns=["Date", "Close"])
    df = df.rename(columns={"Adj Close": "Price"})
    shift = 1
    df["Price T+1"] = df["Price"].shift(-1 * shift)
    print(df.head())
    df.drop(df.tail(shift).index, inplace=True)
    as_array = df.to_numpy().transpose()
    return np.array(as_array[0:5]), as_array[-1]


def build_dataset(data_arrs, target_arr, num_test_cases, seq_length=300):
    X = torch.zeros([num_test_cases, 5, seq_length])
    y = torch.zeros([num_test_cases, 1])
    for example_idx in range(num_test_cases):
        starting_idx = random.randrange(0, len(data_arrs[0]) - seq_length)
        end_idx = starting_idx + seq_length
        random_sub_sequence = np.array([arr[starting_idx:end_idx] for arr in data_arrs])
        if len(random_sub_sequence[0]) != seq_length:
            raise ("incorrect seq length")
        X[example_idx] = torch.from_numpy(random_sub_sequence)
        y[example_idx] = target_arr[end_idx]
    return Variable(X), Variable(y)

In [96]:
data, target = get_data("GOOG.csv")

       Open      High       Low     Price     Volume  Price T+1
0  2.490664  2.591785  2.390042  2.499133  897427216   2.697639
1  2.515820  2.716817  2.503118  2.697639  458857488   2.724787
2  2.758411  2.826406  2.716070  2.724787  366857939   2.611960
3  2.770615  2.779581  2.579581  2.611960  306396159   2.640104
4  2.614201  2.689918  2.587302  2.640104  184645512   2.687676


In [98]:
xs, ys = build_dataset(data, target, 100)

In [101]:
xs[0]

tensor([[6.7181e+00, 6.7211e+00, 6.8874e+00,  ..., 1.3263e+01, 1.3438e+01,
         1.3302e+01],
        [6.9186e+00, 7.0077e+00, 7.0608e+00,  ..., 1.3499e+01, 1.3488e+01,
         1.3395e+01],
        [6.5400e+00, 6.6087e+00, 6.6942e+00,  ..., 1.3239e+01, 1.3327e+01,
         1.3143e+01],
        [6.8521e+00, 6.9597e+00, 6.8329e+00,  ..., 1.3287e+01, 1.3361e+01,
         1.3311e+01],
        [2.3446e+08, 2.3708e+08, 1.9620e+08,  ..., 1.0818e+08, 1.1321e+08,
         1.0738e+08]])

In [102]:
ys[0]

tensor([13.2782])